In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig

In [3]:
import re
def preprocess_text(text):
    # Приведение текста к нижнему регистру
    text = text.lower()
    # Удаление всех цифр
    text = re.sub(r'\d+', '', text)
    # Удаление всех знаков препинания и специальных символов
    text = re.sub(r'[^\w\s]', '', text)
    # Удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [4]:
# Путь к сохраненным файлам модели
model_path = '/home/jupyter/datasphere/project/best_model_bert_with_preprocess2'

In [5]:
# Загрузка конфигурации модели
config = BertConfig.from_pretrained(model_path + '/config.json')

In [6]:
model = BertForSequenceClassification.from_pretrained(
    model_path + '/model.safetensors',
    config=config
)

In [7]:
# Загрузка токенизатора
tokenizer = BertTokenizer.from_pretrained(model_path)

In [8]:
# Перевод модели в режим оценки
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [9]:
# Функция для предсказания
def predict(text, tokenizer, model):
    # Токенизация текста
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    
    # Применение модели
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Вероятности классов
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
    return probabilities.numpy()


In [57]:
# Пример текста для предсказания
text = "няшка няшная жизнь общажная"
text = preprocess_text(text)

In [58]:
# Получение вероятностей классов
probabilities = predict(text, tokenizer, model)

In [59]:
print("Вероятности классов:", probabilities)

Вероятности классов: [[5.4164237e-04 9.9380505e-01 9.2245446e-04 8.8127435e-04 7.0798805e-04
  6.5193040e-04 4.1073564e-04 7.0261210e-04 7.9134188e-04 5.8484962e-04]]


In [60]:
# Получение класса с максимальной вероятностью
predicted_class = probabilities.argmax(axis=-1)
print("Предсказанный класс:", predicted_class)

Предсказанный класс: [1]


In [61]:
# Определение словарей меток
LabelDict = {
    0: 'statement_general',
    1: 'dorm',
    2: 'events',
    3: 'finassist',
    4: 'grade',
    5: 'loc',
    6: 'military',
    7: 'holidays',
    8: 'pay',
    9: 'schedule',
}

In [62]:
print(LabelDict[predicted_class[0]])

dorm
